### Training a binary classifier with the IMDB Reviews Dataset

In [1]:
#importing libraries

import tensorflow as tf
import tensorflow_datasets as tfds
import io


In [2]:
#downloading the dataset

imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
print(info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\mouni\tensorflow_datasets\imdb_reviews\plain_text\incomplete.U78U5P_1.0.0\imdb_reviews-trai…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\mouni\tensorflow_datasets\imdb_reviews\plain_text\incomplete.U78U5P_1.0.0\imdb_reviews-test…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\mouni\tensorflow_datasets\imdb_reviews\plain_text\incomplete.U78U5P_1.0.0\imdb_reviews-unsu…

Dataset imdb_reviews downloaded and prepared to C:\Users\mouni\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_dir='C:\\Users\\mouni\\tensorflow_datasets\\imdb_reviews\\plain_text\\1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dt

In [3]:
print(imdb)

{Split('train'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('test'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('unsupervised'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


In [9]:
#few sample reviews
for element in imdb['train'].take(4):
    print(element)

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

we can see that each example is a 2-element tuple of tensors containing the text first, then the label.

In [10]:
#getting the train and test sets
train_dataset, test_dataset = imdb['train'], imdb['test']

##### Generating padded sequences

we will convert the strings into integer sequences, then pad them to a uniform length.

In [13]:
#parameters

vocab_size = 10000
max_length = 120
embedding_dim = 16
padding_type = 'pre'
trucn_type = 'post'


We will generate vocabulary using training set only, we should not include the test set because that is meant to represent data that the model hasn't seen before.

In [ ]:
#instantiate the vectorization layer

vectorize_layer = tf.keras.layers.TextVectorization(max_tokens = vocab_size)

#getting the review and label for training dataset and testing datasets
train_reviews = train_dataset.map(lambda review, label : review)
test_review = test_dataset.map(lambda review, label : review)

train_labels = train_dataset.map(lambda review, label: label)
test_labels = test_dataset.map(lambda review, label: label)

#generating vocabulary based on training reviews
vectorize_layer.adapt(train_reviews)

